# GeoCLEWs

**Revised code:** [Yalda Saedi](https://github.com/Ysaedi) <br />
**Supervision:** [Taco Niet](https://github.com/tniet) <br />
**Funding** [Mitacs](https://www.mitacs.ca/en) and [Catalyste+](https://www.catalysteplus.org/)

---------------------------

## Summary

GeoCLEWs is a versatile open-source script that offers a wide range of useful features for both developers and users. The script streamlines the detailed land and water processing steps required for Climate Land Energy Water systems (CLEWs) modelling. It is designed to efficiently collect, modify, and process  high-resolution land and water data from Global Agro-ecological Zones (GAEZ v4) database in an automated manner. GeoCLEWs processes agro-climatic potential yield, crop water deficit, crop evapotranspiration, precipitation, and land cover datasets.


This notebook builds upon the [CLEWs GIS Processing notbook](https://github.com/akorkovelos/un-clews-gis-work/blob/main/CLEWs%20GIS%20Processing.ipynb), developed by [Alexandros Korkovelos](https://github.com/akorkovelos), under the supervision of [Abhishek Shivakumar](https://github.com/abhishek0208) & Thomas Alfstad. Please note that the original notebook is currently non-functional due to its incompatibility with the latest version of the GAEZ database. Although the original code served as a valuable starting point, it has undergone significant revisions and improvements:

- Utilizing high-resolution GAEZ v4 datasets to detail land and water systems.
- Implementing customized geographical aggregation and Agglomerative Hierarchical clustering to capture cross-regional interdependencies and streamline computational complexity.
- Automating data collection, preparation, processing, and result generation for diverse geographical regions, reducing manual effort and minimizing human errors in WEF assessment tasks.
- Generating clewsy-compatible outputs.

At the beginning of the script, users can input and tailor the configuration to align with their project's unique requirements. Once the necessary inputs are provided, GeoCLEWs will automatically execute the subsequent steps, including the data collection from FAOSTAT (Food and Agriculture Organization of the United Nation) and retrieving high-resolution raster files from GAEZ v4. Additionally, it implements geographical and spatial clustering to detect cross-regional agro-ecological similarities and generate detailed land and water statistics.

This notebook performs six main analytical processes:

- **Part 1**: Initialization and configuration.
- **Part 2**: FAOSTAT and GAEZ data collection and preparation.
- **Part 3**: Generating land cells.
- **Part 4**: Geospatial attributes extraction to land cells.
- **Part 5**: Spatial clustering using agglomerative hierarchical clustering
- **Part 6**: Calculating key summary statistics generate outputs for further use in CLEWs modelling.

Each part below is accompanied by a more detailed explanation of the involved processing steps.

# Part 1 : Initialization and Configuration

# 1.1. Importing Essential Modules 

To begin, it is necessary to import the required modules/libraries. For more information on the environment setup, please consult the README file. 

In [1]:
# Importing necessary Python modules or libraries
from main import initialize_directories
from libs.constants import USER_INPUTS_PATH
from libs.collect import retrieve_top_10_crops, standardize_faostat, process_gaez_data
from libs.process_land_cells import read_shapefiles, generate_georeference, convert_points_to_polygons, calibrate_area
from libs.extract_geospatial_attributes import clip_raster_file, collect_raster_files, extract_raster_values
from libs.spatial_clustering import prepare_data_for_clustering
from libs.calculation_cluster import calculate

# System & Other
import yaml
import time
import pandas as pd
start_time = time.time()


# 1.2. User Configuration  
This is the only part where the user needs to input values. The rest of the process will be automatically run based on the provided inputs. The code is designed with flexibility, allowing users to make changes and take advantage of customized settings at any time during the execution.


In [2]:
# Provide specifications for the project
with open(f"{USER_INPUTS_PATH}/config.yaml", "r") as f:
    inputs = yaml.load(f, Loader=yaml.FullLoader)
        # 3-letter ISO code of the selected country
    code = pd.read_csv(
        'Data/Country_code.csv')  # More info: https://www.nationsonline.org/oneworld/country_code_list.htm
    code_name = code[code['Full_name'] == inputs["geographic_scope"]]
    inputs["country_name"] = code_name.iloc[0]['country_code']


## 1.3. Directory Initialization and Structure

For easier replication of the code you may used the following directory structure:

* **~root/Data/input**    (a directory where your input data)
* **~root/global_raster_input**   (directory for GAEZ raster layers with global coverage.  Precipitation and land cover rasters have already downloaded in the global_raster_input while agro-climatic potential yield, crop water deficit, and crop evapotranspiration with global coverage will be downloaded automatically in the following steps based on user input)
* **~root/cropped_raster_input** (a directory for cropped GAEZ global raster data based on administraty boundry of selected country to reduce computational complexity)

Results will be store in two automatically generated directories:
* **~root/Data/output**   (directory for general output)
* **~root/Data/output/summary_stats**   (a directory where the tabular outputs data and graphs are stored)

**Note!** In case you decide to use a different structure please revise the code below.

In [3]:
# Directories
initialize_directories(inputs)

# Part 2 : FAOSTAT and GAEZ Data Collection and Preparation

## 2.1. FAOSTAT Collection and Preparation
Finding top 10 crops in terms of harvested area from the latest statistical database provided by Food and Agriculture Organization of the United Nations (FAOSTAT).

### 2.1.1. Retrieve Top 10 Crops

In [4]:
crops_main, crops_others = retrieve_top_10_crops(inputs)

 **Top 5 crops considering harvested area are:** ['Rice', 'Sugar cane', 'Coconuts, in shell', 'Plantains and cooking bananas', 'Maize (corn)']

 **Crops ranked from six to ten in the top 10 FAO dataset are:** ['Ginger, raw', 'Edible roots and tubers with high starch or inulin content, n.e.c., fresh', 'Other tropical fruits, n.e.c.', 'Cassava, fresh', 'Cocoa beans']

### 2.1.2. FAOSTAT Standardizing

In [5]:
crop_name_main, crop_name_other, crop_codes = standardize_faostat(crops_main, crops_others)


 **Based on 3-letter naming, the main crop list from the FAOSTAT is :** ['SGC', 'CON', 'MZE', 'prc']

 **Based on 3-letter naming, additional crop list from the FAOSTAT is :** ['CAS', 'COC']

## 2.2. GAEZ Data Collection and Preparation

GeoCLEWs collects TIFF data from the Global Agro-Ecological Zones data portal for the following variables: agro-climatic potential yield, crop water deficit, and crop evapotranspiration. Precipitation, and land cover have already downloaded in the directory.

In [9]:
process_gaez_data(crop_name_main, crop_name_other, crop_codes, inputs)


# Part 3: Generating Land Cells

## 3.1. Generating Georeferenced Point Grid from Shapefile
Considering the resolution of GAEZ raster files, it is recommended to set spacing to 0.09 decimal degrees resulting in a detailed land and water analysis.

In [11]:
#create a GeoDataFrame from the attributes and geometry of the shapefile
shapefile_admin_lv, shapefile_admin_0 = read_shapefiles(inputs)

In [12]:
#Creating point grid
grid_points = generate_georeference(shapefile_admin_lv, inputs)

## 3.2. Converting Points to Polygons

A regular grid point is created across the entire area of interest in the previous step. Georeferenced points have unique latitude and longitude. In this step,  square buffer-based polygons are created around each point. This allows further flexibility in the extraction of raster values using stats. The buffered polygon shall split "equally" the area between neighbor points; therefore, the buffer used shall be the half of the distance between two neighbor points. This, in turn depends on the location of the AoI on earth and the projection system used. 

### 3.2.1. Spatial Join
Assigning the same administrative region as defined in the GeoDataFrame to the 'cluster' column.

### 3.2.2. Region Aggregation


### 3.2.3. Generating Polygons
Creating Polygons From Georeferenced Clustered Grid Points

In [16]:
gdf_clustered = convert_points_to_polygons(shapefile_admin_lv, grid_points, inputs)


,geometry,index_right,cluster
0,POINT (-61.29692 5.85677),0,GUY
1,POINT (-61.29692 5.94677),0,GUY
2,POINT (-61.29692 6.03677),0,GUY


**Note!** Several features are not classified into a cluster. While points away of the administrative borders will be cut out of the analysis, some points right next to the outer administrative borders might create inconsistency when calculating area. In the following section we are dealing with this problem.

## 3.3. Total Area Re-Estimation & Calibration

This step estimates and calibrates the area (in square km) based on the area provided by the admin layer used in the analysis (e.g. clipping). 

### 3.3.1. Area Calibration

In [21]:
calibrate_area(shapefile_admin_0, gdf_clustered, inputs)

In [31]:
print ("Part 3 complete!")


Part 3 complete!


# Part 4: Geospatial Attributes Extraction to land cells

The functions employed in the fourth part extract values from TIFF-formatted GAEZ raster files, and assign them as attributes to the land cells based on their spatial locations


## 4.1. Clipping GAEZ Raster Files
The administrative boundary of the study area is used to clip the GAEZ raster files with global coverage, which leads to a reduction in the computational processing time

In [32]:
clip_raster_file(shapefile_admin_lv, inputs)

## 4.2. Collecting Raster Files

In [36]:
raster_con, raster_dis = collect_raster_files()

We have identified 67 continuous raster(s): 

* MZE cwd Rain-fed Low.tif
* CAS yld Rain-fed High.tif
* COC evt Rain-fed Low.tif
* CON evt Irrigation High.tif
* CON evt Rain-fed High.tif
* CAS evt Rain-fed High.tif
* CON evt Irrigation Low.tif
* SGC cwd Rain-fed High.tif
* MZE evt Irrigation Low.tif
* CAS yld Irrigation High.tif
* COC cwd Irrigation Low.tif
* COC cwd Irrigation High.tif
* SGC evt Irrigation High.tif
* CON yld Rain-fed High.tif
* MZE cwd Irrigation High.tif
* SGC yld Irrigation Low.tif
* SGC yld Irrigation High.tif
* CON cwd Irrigation High.tif
* CAS evt Irrigation High.tif
* CAS cwd Rain-fed Low.tif
* MZE evt Rain-fed Low.tif
* COW yld Irrigation High.tif
* MZE cwd Rain-fed High.tif
* SGC yld Rain-fed Low.tif
* CAS evt Rain-fed Low.tif
* TEA yld Rain-fed High.tif
* COW yld Rain-fed High.tif
* SGC cwd Irrigation Low.tif
* CON yld Irrigation High.tif
* COC evt Irrigation High.tif
* CAS yld Irrigation Low.tif
* SGC evt Rain-fed High.tif
* CON cwd Rain-fed Low.tif
* COC evt

## 4.3. Extracting Raster Values

In [42]:
extract_raster_values(inputs, raster_con, raster_dis)

Part 4 complete!


# Part 5: Spatial Clustering

The study area is spatially clustered based on similarities in agro-ecological potential yield using the agglomerative hierarchical clustering method 

In [43]:
land_cell, region_info = prepare_data_for_clustering(inputs)

 List of regions:['GUY']

# Part 6: Statistics Calculation

This part calculates summary statistics for the generated clusters. There outputs include:

* Tabular summaries (.csv format) grouped by cluster

## 6.1. Calculating Cluster Summaries

In [64]:
calculate(land_cell, crop_name_other, region_info, inputs)


Part 6 - and with that the analysis - completed!
Total elapsed time: 00:13:10
